# Median Return

## 本段代码展示了利用quantOS系统提取数据，并计算指数年收益率及指数成份股年收益率中位数的具体步骤。
## 在程序运行之前，需要您在环境设置中更改以下参数：
## 1. 将YourPhoneNo.改为您在quantOS网站注册的手机号;
## 2. 将YourToken改为您的token;

## 1. 环境设置

In [5]:
from jaqs.data.dataservice import RemoteDataService
from jaqs.data.dataview import DataView
import pandas as pd
import numpy as np

In [6]:
# 设置服务器地址、用户名密码
# 例如：
# data_config = {
#   "remote.data.address": "tcp://data.tushare.org:8910",
#   "remote.data.username":  '18688888888',
#   "remote.data.password":  '23sdjfk209d0fs9dejkl2j3k4j9d0fsdf'}

phone = "xxxx"
token = "xxxxxxxxxx"

data_config = {
  "timeout": 1800,
  "remote.data.address": "tcp://data.tushare.org:8910",
  "remote.data.username":  phone,
  "remote.data.password":  token}

# 启动DataView，并设置用户名密码
dv = DataView()
ds = RemoteDataService()
ds.init_from_config(data_config)


Begin: DataApi login 17321165656@tcp://data.tushare.org:8910
    login success 



'0,'

## 2. 参数设置 

In [7]:
# -------------------------------------------------------------------------------
# 设置待分析版块及分析起止日期
# -------------------------------------------------------------------------------
# 上证50， 沪深300，中证500，中证全指
# indexList = ['000016.SH', '000300.SH', '000905.SH', '000985.CSI']
indexList = ['000016.SH', '000300.SH', '000905.SH', '000985.CSI']
retCompare = pd.DataFrame()
startdate = 20121231
enddate = 20171130

## 3. 分析步骤 (注：中证全指000985.CSI中成份股较多，此部分程序运行之间较长)

In [5]:
for index in indexList:

    # 设置DataView
    props = {'start_date': startdate, 'end_date': enddate, 'universe': index,
             'fields': 'close_adj',
             'freq': 1}
    dv.init_from_config(props, data_api=ds)
    dv.prepare_data()

    print(index)

    # 取出板块指数，版块成份股日收盘价及是否为成分股的信息
    closeData = dv.get_ts('close_adj')
    closeData['trade_date'] = closeData.index
    closeData['trade_date'] = closeData['trade_date'].apply(lambda x: str(x))
    closeData['trade_y'] = closeData['trade_date'].apply(lambda x: x[:4])

    benchmarkData = dv.data_benchmark
    benchmarkData['trade_date'] = benchmarkData.index
    benchmarkData['trade_date'] = benchmarkData['trade_date'].apply(lambda x: str(x))
    benchmarkData['trade_y'] = benchmarkData['trade_date'].apply(lambda x: x[:4])

    isIndexMember = dv.get_ts('index_member') == 1
    isIndexMember['trade_date'] = isIndexMember.index
    isIndexMember['trade_date'] = isIndexMember['trade_date'].apply(lambda x: str(x))
    isIndexMember['trade_y'] = isIndexMember['trade_date'].apply(lambda x: x[:4])

    # 筛选出成份股及指数年末收盘价
    closeData_Y = closeData.groupby(['trade_y']).last()
    closeData_Y = closeData_Y.drop('trade_date', 1)
    benchmark_Y = benchmarkData.groupby(['trade_y']).last()
    benchmark_Y = benchmark_Y.drop('trade_date', 1)
    isIndexMember_Y = isIndexMember.groupby(['trade_y']).last()
    isIndexMember_Y = isIndexMember_Y.drop('trade_date', 1)
    isIndexMember_Y = isIndexMember_Y.iloc[:-1, :]

    # 计算成份股及指数年收益率
    ret_Y = closeData_Y.diff() / closeData_Y.shift(1) * 100
    ret_Y_benchmark = benchmark_Y.diff() / benchmark_Y.shift(1) * 100

    ret_Y = ret_Y.iloc[1:,:]
    ret_Y_benchmark = ret_Y_benchmark.iloc[1:,:]

    ret_Y_satisfied = pd.DataFrame(ret_Y.values * isIndexMember_Y.values)

    # 计算成份股年收益率中位数
    ret_Y_satisfied = ret_Y_satisfied.replace(0.0, np.nan)
    ret_Y_satisfied['median_ret'] = ret_Y_satisfied.median(axis = 1)

    outputRet = {'index_ret': ret_Y_benchmark['close'].tolist(), 'median_ret': ret_Y_satisfied['median_ret'].tolist()}
    outputRet = pd.DataFrame.from_dict(outputRet)
    outputRet.index = ret_Y_benchmark.index
    
    retCompare = pd.concat([retCompare, outputRet], axis = 1)

Initialize config success.
Query data...
Query data - query...
NOTE: price adjust method is [post adjust]
Query data - daily fields prepared.
Query instrument info...
Query adj_factor...
Query benchmark...
Query benchmar member info...
Data has been successfully prepared.
000016.SH
Initialize config success.
Query data...
Query data - query...
NOTE: price adjust method is [post adjust]
Query data - daily fields prepared.
Query instrument info...
Query adj_factor...
Query benchmark...
Query benchmar member info...
Data has been successfully prepared.
000300.SH
Initialize config success.
Query data...
Query data - query...
NOTE: price adjust method is [post adjust]
Query data - daily fields prepared.
Query instrument info...
Query adj_factor...
Query benchmark...
Query benchmar member info...
Data has been successfully prepared.
000905.SH
Initialize config success.
Query data...
Query data - query...
NOTE: price adjust method is [post adjust]
Query data - daily fields prepared.
Query ins

In [6]:
retCompare.columns = ['SZ50_index', 'SZ50_median', 'HS300_index', 'HS300_median', 'ZZ500_index', 'ZZ500_median', 'Ashare_index', 'Ashare_median']
retCompare.T

trade_y,2013,2014,2015,2016,2017
SZ50_index,-15.228618,63.931810,-6.227466,-5.531327,25.210936
SZ50_median,-15.230643,59.908578,-3.185420,-7.682435,10.946135
HS300_index,-7.646836,51.659466,5.583366,-11.281784,21.027236
HS300_median,-6.487704,44.434358,12.456008,-15.269830,1.906094
ZZ500_index,16.888268,39.007066,43.116632,-17.775252,-0.003266
ZZ500_median,10.059198,33.139987,39.477668,-18.092379,-9.616558
Ashare_index,5.213341,45.824358,32.558328,-14.405746,2.647241
Ashare_median,15.837937,36.908015,60.829493,-14.875100,-18.338202
